# Section 5: Sampler Primitive (12% of Exam)

> **Exam Weight**: ~8 questions | **Difficulty**: Medium-High | **Must Master**: ✅✅✅

---

## 📖 Overview

**Sampler** is the NEW way (Qiskit 1.0+) to get measurement statistics from quantum circuits. Understanding Sampler is ABSOLUTELY CRITICAL for the certification exam!

```
Old Way (Deprecated):              New Way (Qiskit 1.0+):
    execute()                          Sampler
    Aer.get_backend()                  
    backend.run()   
❌ DON'T use these anymore!        ✅ Use Sampler Primitive!
```

### What You'll Learn

- Get measurement statistics (counts, probabilities)
- SamplerV2 API and PUB (Primitive Unified Blocs) format
- Multi-circuit execution
- Result extraction patterns

---

## 🎯 Core Concepts

### What are Primitives?

**Primitives** are high-level computational building blocks that provide a simplified interface for running quantum computations.

```
Primitives = Simplified quantum execution interface

Why Primitives?
✓ Hardware-agnostic (same code for simulator/hardware)
✓ Auto-transpilation
✓ Error mitigation built-in
✓ Cleaner, more Pythonic API
✓ Future-proof (IBM's long-term direction)

Two Core Primitives:
• Sampler  → Measurement outcomes (bitstrings, counts)
• Estimator → Expectation values (observables)
```

### 🧠 Analogy: The Loaded Die

Using a `Sampler` is like rolling a loaded die to figure out its bias:
- **Circuit**: The manufacturing process of the die
- **Shots**: How many times you roll it (e.g., 1000 times)
- **Quasi-Probabilities**: Mathematical correction for error mitigation

### Visual: Sampler Workflow

```
┌─────────────────────────────────────────────────────┐
│                 Your Circuit                         │
│     ┌───┐                                           │
│ q: ─┤ H ├──■────M                                   │
│     └───┘┌─┴─┐  │                                   │
│ q: ──────┤ X ├──M                                   │
│          └───┘                                       │
└─────────────────────────────────────────────────────┘
                    │
               ┌────▼─────┐
               │ Sampler  │
               └────┬─────┘
                    │
               ┌────▼────────┐
               │ Measurement │
               │   Counts    │
               │ {'00':512,  │
               │  '11':512}  │
               └─────────────┘
```

**Two Types**:
1. **StatevectorSampler** - Local simulation (fast, perfect)
2. **Sampler** (Runtime) - Real IBM hardware (queue, noise)

**CRITICAL**: Circuit MUST have measurements!

### ⚠️ EXAM TRAP: Sampler vs Estimator

| Feature | Sampler | Estimator |
|---------|---------|------------|
| **Purpose** | Measurement counts | Expectation values |
| **Needs measurements?** | ✅ YES | ❌ NO |
| **Output** | `{'00': 512, '11': 512}` | `⟨H⟩ = 0.73` |
| **Use case** | Get bitstrings | Calculate energy |

**Memory Aid**: "Sampler = Sample bitstrings, Estimator = Estimate ⟨O⟩"

### 🔄 When to Use Sampler vs Estimator?

```
┌────────────────────────────────────────────┐
│         Use Sampler When:                  │
├────────────────────────────────────────────┤
│ • You want to measurement counts           │
│ • Need to sample output distribution       │
│ • Testing classical output                 │
│ • Debugging circuits                       │
│ • Grover's algorithm (find marked state)   │
│ • Quantum Machine Learning classification  │
└────────────────────────────────────────────┘

┌────────────────────────────────────────────┐
│         Use Estimator When:                │
├────────────────────────────────────────────┤
│ • You need expectation values ⟨ψ|O|ψ⟩     │
│ • VQE, QAOA optimization                   │
│ • Quantum chemistry calculations           │
│ • Observable measurements                  │
└────────────────────────────────────────────┘

EXAM TIP:
- Counts/Probabilities → Sampler (Section 5)
- Expectation values → Estimator (Section 6)
```

## Part 2: Local Sampler - Basic Usage

In [ ]:
# Basic Sampler Pattern - EXECUTABLE

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler

# Create circuit with measurements
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])  # MUST have measurements!

print("Circuit:")
print(qc.draw())

# Create Sampler
sampler = StatevectorSampler()

# Run circuit (note: PUB format with tuple!)
job = sampler.run([(qc,)], shots=1024)

# Get results
result = job.result()
counts = result[0].data.c.get_counts()

print(f"\nMeasurement results: {counts}")
print(f"Total shots: {sum(counts.values())}")

print("\n✓ StatevectorSampler for local simulation")
print("✓ result[0].data.meas.get_counts() for counts")
print("✓ Circuit MUST have measurements")

### 🎯 EXAM CRITICAL: Result Access Pattern

**MEMORIZE THIS**:
```python
counts = result[0].data.c.get_counts()
          ↑      ↑    ↑       ↑
          │      │    │       └─ Method
          │      │    └───────── Measurement register
          │      └───────────── Data attribute
          └─────────────────── Circuit index
```

## Part 3: Multiple Circuits with Sampler

In [ ]:
# Multiple Circuits Pattern - EXECUTABLE

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler as Sampler

# Create three circuits
qc1 = QuantumCircuit(2, 2)  # Bell |Φ+⟩
qc1.h(0)
qc1.cx(0, 1)
qc1.measure([0, 1], [0, 1])

qc2 = QuantumCircuit(2, 2)  # Bell |Ψ+⟩
qc2.h(0)
qc2.x(1)
qc2.cx(0, 1)
qc2.measure([0, 1], [0, 1])

qc3 = QuantumCircuit(3, 3)  # GHZ
qc3.h(0)
qc3.cx(0, 1)
qc3.cx(0, 2)
qc3.measure([0, 1, 2], [0, 1, 2])

# Run all together (correct PUB format: list of tuples)
sampler = Sampler()
pubs = [(qc1,), (qc2,), (qc3,)]
job = sampler.run(pubs, shots=1024)
# pubs = [(qc1, None, 1024), (qc2, None, 1024), (qc3, None, 1024)]
# job = sampler.run(pubs)
result = job.result()

# Access each result
counts1 = result[0].data.c.get_counts()
counts2 = result[1].data.c.get_counts()
counts3 = result[2].data.c.get_counts()

print("Bell |Φ+⟩ (qc1):", counts1)
print("Bell |Ψ+⟩ (qc2):", counts2)
print("GHZ (qc3):      ", counts3)

print("\n✓ Pass list of circuits to run()")
print("✓ result[i] for each circuit")
print("✓ More efficient than separate jobs")

## Part 4: Parameterized Circuits

In [ ]:
# Parameterized Circuits - EXECUTABLE

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.primitives import StatevectorSampler as Sampler
import numpy as np

# Create parameterized circuit
theta = Parameter('θ')
qc = QuantumCircuit(1, 1)
qc.ry(theta, 0)
qc.measure(0, 0)

print("Parameterized circuit:")
print(qc.draw())

# Bind parameter and run single
sampler = Sampler()
pub = [(qc, [np.pi/4], 1024)]
job = sampler.run(pub)
result = job.result()
counts = result[0].data.c.get_counts()
print(f"\nθ = π/4: {counts}")

# Multiple parameter values
angles = [0, np.pi/4, np.pi/2, np.pi]
pub_multi = [(qc, angles, 1024)]
job_multi = sampler.run(pub_multi)
results_multi = job_multi.result()

print("\nParameter sweep results:")
# When passing multiple parameter values, result[0].data.c is a BitArray with shape
bit_array = results_multi[0].data.c
for i, angle in enumerate(angles):
    counts = bit_array.get_counts(i)  # Use index i to get counts for each parameter
    p0 = counts.get('0', 0) / 1024
    print(f"θ = {angle:.2f}: P(|0⟩) = {p0:.3f}")

print("\n✓ Must bind parameters BEFORE Sampler")
print("✓ Use assign_parameters()")
print("✓ Can bind multiple values and run together")

### ⚠️ EXAM TRAP: Parameter Binding

```python
# WRONG - Don't pass parameterized circuit
qc = QuantumCircuit(1)
qc.ry(Parameter('θ'), 0)
sampler.run([qc])  # ❌ ERROR!

# CORRECT - Bind first
qc_bound = qc.assign_parameters({'θ': 0.5})
sampler.run([qc_bound])  # ✅
```

## Part 5: Result Processing

In [ ]:
# Result Processing - EXECUTABLE

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler as Sampler

# Create Bell state circuit
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

# Run
sampler = Sampler()
pub = [(qc, None, 1024)]
job = sampler.run(pub)
result = job.result()

# Get counts
counts = result[0].data.c.get_counts()
print(f"Raw counts: {counts}")

# Convert to probabilities
total_shots = sum(counts.values())
probs = {state: count/total_shots for state, count in counts.items()}
print(f"\nProbabilities: {probs}")

# Most probable state
most_probable = max(counts, key=counts.get)
print(f"\nMost probable: |{most_probable}⟩")

# Success rate for target
target = '00'
success_rate = counts.get(target, 0) / total_shots
print(f"P(|{target}⟩) = {success_rate:.1%}")

# Sort by probability
sorted_probs = sorted(probs.items(), key=lambda x: x[1], reverse=True)
print(f"\nSorted by probability: {sorted_probs}")

print("\n" + "="*50)
print("ADVANCED: Raw Data Access")
print("="*50)

pub_result = result[0]

# Get bit array object
bitarray = pub_result.data.c
print(f"\nBitArray type: {type(bitarray)}")
print(f"BitArray shape: {bitarray.shape}")

# Access metadata
metadata = pub_result.metadata
print(f"Metadata: {metadata}")

## Part 6: Runtime Sampler Pattern

In [ ]:
# Runtime Sampler Pattern (Real Hardware)
# This is reference code - requires IBM Quantum account

print("""
Runtime Sampler Pattern (for real hardware):
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, Session
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# Setup service
service = QiskitRuntimeService()
backend = service.least_busy(simulator=False, operational=True)

# Create circuit
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

# Transpile for target backend
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)
qc_transpiled = pm.run(qc)

# Run with Session
with Session(backend=backend) as session:
    sampler = Sampler(mode=session)
    job = sampler.run([qc_transpiled], shots=4096)
    result = job.result()
    counts = result[0].data.meas.get_counts()
""")

print("\n🎯 Key Differences from Local:")
print("  ✓ Import from qiskit_ibm_runtime")
print("  ✓ Need QiskitRuntimeService and backend")
print("  ✓ Transpile circuit before running")
print("  ✓ Use Session for efficiency")
print("  ✓ Same result access: result[0].data.meas.get_counts()")

## Part 7: Local vs Runtime Comparison

In [ ]:
# Comparison table
print("Local vs Runtime Sampler:")
print("""
┌────────────────────┬─────────────────────┬──────────────────────┐
│ Feature            │ Local               │ Runtime              │
├────────────────────┼─────────────────────┼──────────────────────┤
│ Import from        │ qiskit.primitives   │ qiskit_ibm_runtime   │
│ Runs on            │ Your computer       │ IBM hardware         │
│ Requires account   │ No                  │ Yes                  │
│ Speed              │ Instant             │ Queue time           │
│ Noise              │ None (perfect)      │ Real hardware noise  │
│ Usage              │ Testing/dev         │ Production           │
│ Sessions           │ No                  │ Yes (recommended)    │
│ Options            │ Basic               │ Advanced             │
│ Result access      │ Same for both       │ Same for both        │
└────────────────────┴─────────────────────┴──────────────────────┘
""")

print("\n✓ Both use: result[i].data.meas.get_counts()")
print("✓ Local for testing, Runtime for production")

### Import Statements

```python
# Local Sampler (simulation)
from qiskit.primitives import StatevectorSampler as Sampler
sampler = Sampler()

# Runtime Sampler (real hardware)
from qiskit_ibm_runtime import Sampler
sampler = Sampler(backend=backend)
```

## Part 7.5: 💡 Practical Patterns

### Use Cases for Sampler in Real Applications

In [ ]:
# Pattern 1: Circuit Debugging with Sampler - EXECUTABLE

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler

print("Pattern 1: Circuit Debugging")
print("="*50)

# Create GHZ circuit to verify
qc = QuantumCircuit(3, 3)
qc.h(0)
qc.cx(0, 1)
qc.cx(0, 2)  # Intended GHZ state
qc.measure([0, 1, 2], [0, 1, 2])

print("GHZ Circuit:")
print(qc.draw())

# Sample to verify
sampler = StatevectorSampler()
pubs = [(qc, None, 1000)]
job = sampler.run(pubs)
counts = job.result()[0].data.c.get_counts()

print(f"\nMeasurement results: {counts}")
print("\nExpected: ~50% |000⟩, ~50% |111⟩")

# Verify GHZ state
p_000 = counts.get('000', 0) / 1000
p_111 = counts.get('111', 0) / 1000
is_ghz = p_000 > 0.4 and p_111 > 0.4

if is_ghz:
    print("✓ GHZ state verified!")
else:
    print("✗ Not a valid GHZ state - check circuit!")

print("\n✓ Use Sampler to verify circuit behavior")
print("✓ Expected vs actual distribution reveals bugs")

In [ ]:
# Pattern 2: Multi-Circuit Comparison - EXECUTABLE

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler

print("Pattern 2: Multi-Circuit Comparison")
print("="*50)

# Create variations to compare
circuits = []
for n_layers in [1, 2, 3, 4]:
    qc = QuantumCircuit(3, 3)
    for _ in range(n_layers):
        qc.h([0, 1, 2])
        qc.cx(0, 1)
        qc.cx(1, 2)
    qc.measure([0, 1, 2], [0, 1, 2])
    # circuits.append((qc,))  # Note: tuple format!
    circuits.append((qc, None, 1024))  # Note: tuple format!

# Run all at once (more efficient!)
sampler = StatevectorSampler()
# job = sampler.run(circuits, shots=1024)
job = sampler.run(circuits)
results = job.result()

# Analyze each
print("\nResults by circuit depth:")
for i, result in enumerate(results):
    counts = result.data.c.get_counts()
    n_states = len(counts)
    max_state = max(counts, key=counts.get)
    max_prob = counts[max_state] / 1024
    print(f"Layers {i+1}: {n_states} unique states, max P(|{max_state}⟩) = {max_prob:.3f}")

print("\n✓ Batch multiple circuits for efficiency")
print("✓ Compare different circuit depths")
print("✓ Each result indexed by circuit order")

In [ ]:
# Pattern 3: Probability Distribution Analysis - EXECUTABLE

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler
import statistics

print("Pattern 3: Probability Distribution Analysis")
print("="*50)

# Create equal superposition
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.h(1)
qc.measure([0, 1], [0, 1])

print("Equal superposition circuit:")
print(qc.draw())

# Sample
sampler = StatevectorSampler()
job = sampler.run([(qc,)], shots=4096)
counts = job.result()[0].data.c.get_counts()

# Convert to probabilities
total = sum(counts.values())
probs = {state: count/total for state, count in counts.items()}

print("\nProbability distribution:")
for state, prob in sorted(probs.items()):
    bar = '█' * int(prob * 40)
    print(f"|{state}⟩: {prob:.4f} {bar}")

# Expected: ~0.25 for each of |00⟩, |01⟩, |10⟩, |11⟩

# Statistical analysis
prob_values = list(probs.values())
mean_prob = statistics.mean(prob_values)
std_prob = statistics.stdev(prob_values) if len(prob_values) > 1 else 0

print(f"\nStatistical Analysis:")
print(f"  Mean probability: {mean_prob:.4f} (expected: 0.25)")
print(f"  Std deviation: {std_prob:.4f} (expected: ~0)")

print("\n✓ Analyze probability distributions from counts")
print("✓ Verify uniform distributions (superposition)")
print("✓ Use statistics for deeper analysis")

## Part 8: PUB Format (EXAM CRITICAL!)

### What is PUB?

**PUB** = Primitive Unified Bloc = (circuit, parameters, shots)

**CRITICAL**: Must be tuple inside list!

In [ ]:
# PUB Format Examples - EXECUTABLE

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler as Sampler

# Create test circuit
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

sampler = Sampler()

print("PUB Format Examples:")
print("="*50)

# Single circuit - CORRECT format with trailing comma
print("\n1. Single circuit (tuple in list with trailing comma):")
job = sampler.run([(qc,)])
counts = job.result()[0].data.c.get_counts()
print(f"   Result: {counts}")

# Multiple circuits
qc2 = QuantumCircuit(1, 1)
qc2.x(0)
qc2.measure(0, 0)

print("\n2. Multiple circuits:")
job = sampler.run([(qc,), (qc2,)])
result = job.result()
print(f"   Circuit 1: {result[0].data.c.get_counts()}")
print(f"   Circuit 2: {result[1].data.c.get_counts()}")

# With custom shots
print("\n3. With custom shots (1000):")
job = sampler.run([(qc,)], shots=1000)
counts = job.result()[0].data.c.get_counts()
print(f"   Total shots: {sum(counts.values())}")

print("\n🎯 EXAM TIP: PUB = Tuple in List!")
print("  ✓ [(circuit,)] - Note trailing comma")
print("  ✓ Memory: CPS = Circuit, Parameters, Shots")

### ⚠️ EXAM TRAP: Trailing Comma

```python
(qc)     # NOT a tuple! Just parentheses
(qc,)    # ✓ Tuple with one element

# In Python:
type((qc))   # <class 'QuantumCircuit'>
type((qc,))  # <class 'tuple'>
```

**Memory Aid**: "Tuple needs trailing comma, like a list with one item!"

### PUB Format Complete Reference

| Scenario | PUB Format | Example |
|----------|------------|----------|
| Single circuit | `[(circuit,)]` | `sampler.run([(qc,)])` |
| With parameters | `[(circuit, params)]` | `sampler.run([(qc, [0.5, 1.2])])` |
| Multiple circuits | `[(qc1,), (qc2,)]` | `sampler.run([(qc1,), (qc2,)])` |
| Custom shots | `[(circuit, None, shots)]` | `sampler.run([(qc, None, 2048)])` |
| Everything custom | `[(circuit, params, shots)]` | `sampler.run([(qc, [0.5], 4096)])` |

## Part 9: 🎯 Practice Problems

### Practice coding problems to solidify Sampler concepts

In [ ]:
# Practice Problem 1: Create and Measure GHZ State
print("Practice Problem 1: GHZ State Measurement")
print("="*50)

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler

# Create GHZ state: (|000⟩ + |111⟩)/√2
qc = QuantumCircuit(3, 3)
qc.h(0)
qc.cx(0, 1)
qc.cx(1, 2)
qc.measure([0, 1, 2], [0, 1, 2])

print("Circuit:")
print(qc.draw())

# Sample
sampler = StatevectorSampler()
job = sampler.run([(qc,None, 1000)])
counts = job.result()[0].data.c.get_counts()

print(f"\nResults: {counts}")
print("Expected: ~500 '000' and ~500 '111' (entangled!)")

In [ ]:
# Practice Problem 2: W-State Verification
print("Practice Problem 2: W-State Verification")
print("="*50)

from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler
import math

# Create W-state: (|100⟩ + |010⟩ + |001⟩)/√3
# This is a 3-qubit entangled state with exactly one qubit in |1⟩
qc = QuantumCircuit(3, 3)
qc.ry(2*math.acos(math.sqrt(1/3)), 0)
qc.ch(0, 1)
qc.x(0)
qc.ccx(0, 1, 2)
qc.x(0)
qc.measure([0, 1, 2], [0, 1, 2])

print("Circuit for W-state:")
print(qc.draw())

# Sample
sampler = StatevectorSampler()
job = sampler.run([(qc,)], shots=3000)
counts = job.result()[0].data.c.get_counts()

print(f"\nResults: {counts}")
print("Expected: ~1000 each for '001', '010', '100' (W-state!)")

# Verify probability
total = sum(counts.values())
for state, count in sorted(counts.items()):
    print(f"P(|{state}⟩) = {count/total:.3f}")

In [ ]:
# Practice Problem 3: Parameterized Circuit Sweep
print("Practice Problem 3: Parameterized Circuit Sweep")
print("="*50)

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.primitives import StatevectorSampler
import numpy as np

# Parameterized circuit: Ry(θ)|0⟩
theta = Parameter('θ')
qc = QuantumCircuit(1, 1)
qc.ry(theta, 0)
qc.measure(0, 0)

print("Parameterized circuit:")
print(qc.draw())

# Sweep parameter from 0 to π
sampler = StatevectorSampler()
angles = np.linspace(0, np.pi, 5)

# Create PUBs with different parameter values
pubs = [(qc, [angle]) for angle in angles]
job = sampler.run(pubs, shots=1024)
results = job.result()

print("\nParameter sweep results:")
print(f"{'θ':^10} | {'P(|0⟩)':^10} | {'P(|1⟩)':^10} | {'Theory P(|1⟩)':^12}")
print("-"*50)

for i, angle in enumerate(angles):
    counts = results[i].data.c.get_counts()
    total = sum(counts.values())
    p0 = counts.get('0', 0) / total
    p1 = counts.get('1', 0) / total
    theory = np.sin(angle/2)**2  # P(|1⟩) = sin²(θ/2) for Ry(θ)|0⟩
    print(f"{angle:^10.3f} | {p0:^10.3f} | {p1:^10.3f} | {theory:^12.3f}")

print("\n✓ Ry(θ)|0⟩ → P(|1⟩) = sin²(θ/2)")
print("✓ At θ=0: all |0⟩, at θ=π: all |1⟩")

## Part 10: ✅ Pre-Flight Checklist

### Before Using Sampler - Quick Validation

```
□ Circuit has measure() or measure_all()?
□ Using correct PUB format: [(circuit,)] with trailing comma?
□ Parameters bound with assign_parameters()?
□ Correct import: StatevectorSampler for local, Sampler for runtime?
□ Accessing results: result[0].data.meas.get_counts()?
□ Handling multiple circuits: result[i] for each?
□ Shots parameter: Default 1024, or custom in PUB?
```

### Common Error Prevention

| Error Type | Symptom | Fix |
|------------|---------|-----|
| No measurements | Error on run | Add `qc.measure()` or `qc.measure_all()` |
| Wrong PUB format | Type error | Use `[(qc,)]` with trailing comma |
| Unbound parameters | Parameter error | Call `qc.assign_parameters()` first |
| Wrong result access | AttributeError | Use `result[0].data.meas.get_counts()` |
| Wrong import | ImportError | Check local vs runtime imports |

## 📝 Practice Questions

### Question 1: Result Access

**How do you extract measurement counts from Sampler results?**

A) `result.data.meas.get_counts()`  
B) `result[0].data.meas.get_counts()`  
C) `result[0].counts()`  
D) `result[0].get_counts()`

<details>
<summary>Answer</summary>

**B) `result[0].data.meas.get_counts()`**

```python
result = job.result()
counts = result[0].data.meas.get_counts()
          ↑      ↑    ↑       ↑
          │      │    │       └─ Method
          │      │    └───────── Register name
          │      └───────────── Data attribute
          └─────────────────── Circuit index
```

**MEMORIZE THIS PATTERN!**
</details>

---

### Question 2: PUB Format

**Which is the correct PUB format for single circuit?**

A) `sampler.run(qc)`  
B) `sampler.run([qc])`  
C) `sampler.run([(qc)])`  
D) `sampler.run([(qc,)])`

<details>
<summary>Answer</summary>

**D) `sampler.run([(qc,)])`**

```python
# Correct: Tuple inside list
sampler.run([(qc,)])  # ✓

# Wrong patterns:
sampler.run(qc)       # ❌ Not wrapped
sampler.run([qc])     # ❌ Not tuple
sampler.run([(qc)])   # ❌ Missing trailing comma
```

**Key**: Trailing comma makes it a tuple!
</details>

---

### Question 3: Measurements Required?

**Does Sampler require measurements in the circuit?**

A) No, Sampler adds them automatically  
B) Yes, circuit MUST have measure()  
C) Only for real hardware  
D) Depends on backend

<details>
<summary>Answer</summary>

**B) Yes, circuit MUST have measure()**

```python
# WRONG - No measurements
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
sampler.run([(qc,)])  # ❌ ERROR!

# CORRECT - With measurements
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])  # ✓ Added
sampler.run([(qc,)])  # ✓ Works
```

**Note**: Estimator does NOT need measurements!
</details>

---

### Question 4: Local vs Runtime Import

**How do you import local Sampler for simulation?**

A) `from qiskit_ibm_runtime import Sampler`  
B) `from qiskit.primitives import Sampler`  
C) `from qiskit.primitives import StatevectorSampler as Sampler`  
D) `from qiskit import Sampler`

<details>
<summary>Answer</summary>

**C) `from qiskit.primitives import StatevectorSampler as Sampler`**

```python
# Local Sampler (simulation)
from qiskit.primitives import StatevectorSampler as Sampler
sampler = Sampler()

# Runtime Sampler (real hardware)
from qiskit_ibm_runtime import Sampler
sampler = Sampler(backend=backend)
```

**Key**: qiskit.primitives for local, qiskit_ibm_runtime for hardware
</details>

---

### Question 5: Multiple Circuit Results

**How do you access results from the third circuit in a batch run?**

A) `result[3].data.meas.get_counts()`  
B) `result[2].data.meas.get_counts()`  
C) `result.data[2].meas.get_counts()`  
D) `result[2].meas.get_counts()`

<details>
<summary>Answer</summary>

**B) `result[2].data.meas.get_counts()`**

```python
# Run multiple circuits
job = sampler.run([(qc1,), (qc2,), (qc3,)])
result = job.result()

# Access third circuit (0-indexed!)
counts = result[2].data.meas.get_counts()
          ↑
          └─ Index 2 = third circuit (0-indexed)
```

**Key**: Python uses 0-based indexing!
</details>

---

## 📋 Sampler API Cheat Sheet

### Quick Reference Card

```python
# ═══════════════════════════════════════════════════════════
#                 SAMPLER QUICK REFERENCE
# ═══════════════════════════════════════════════════════════

# IMPORTS
from qiskit.primitives import StatevectorSampler  # Local simulation
from qiskit_ibm_runtime import SamplerV2 as Sampler  # Hardware

# CREATE
sampler = StatevectorSampler()  # or Sampler(backend=backend)

# RUN (PUB format!)
job = sampler.run([(qc,)], shots=1024)  # Note tuple with comma!

# RESULTS
result = job.result()
counts = result[0].data.meas.get_counts()

# ═══════════════════════════════════════════════════════════
```

### PUB Format Reference Table

| Scenario | PUB Format | Example |
|----------|------------|---------|
| Single circuit | `[(circuit,)]` | `sampler.run([(qc,)])` |
| With parameters | `[(circuit, params)]` | `sampler.run([(qc, [0.5, 1.2])])` |
| Multiple circuits | `[(qc1,), (qc2,)]` | `sampler.run([(qc1,), (qc2,)])` |
| Custom shots | `[(circuit, None, shots)]` | `sampler.run([(qc, None, 2048)])` |
| Everything custom | `[(circuit, params, shots)]` | `sampler.run([(qc, [0.5], 4096)])` |

---

## ✅ Key Takeaways

### Core Concepts

1. **Sampler Purpose**
   - Get measurement counts/probabilities
   - Samples from quantum state
   - Circuit MUST have measurements

2. **Two Types**
   - StatevectorSampler: Local simulation (ideal)
   - SamplerV2/Sampler (Runtime): Real hardware (noisy)
   - Same result access pattern for both

3. **Result Access**
   - `result[i].data.meas.get_counts()`
   - Returns dict: `{'00': 512, '11': 512}`
   - Index [i] for multiple circuits (0-indexed)

4. **PUB Format**
   - Tuple inside list: `[(circuit,)]`
   - Trailing comma required!
   - Order: (circuit, parameters, shots) = CPS

### 🚨 Critical Exam Facts

| Fact | Details |
|------|---------|
| Result Pattern | `result[0].data.meas.get_counts()` |
| Measurement Requirement | Circuit MUST have `measure()` or `measure_all()` |
| PUB Format | `[(circuit,)]` with trailing comma |
| Local Import | `from qiskit.primitives import StatevectorSampler` |
| Runtime Import | `from qiskit_ibm_runtime import SamplerV2 as Sampler` |
| Parameter Binding | Must use `assign_parameters()` BEFORE running |
| Multiple Circuits | Pass list of PUBs, access `result[i]` |
| Default Shots | 1024 (can override per PUB) |

### Common Traps & Fixes

| ❌ Trap | ✅ Fix |
|---------|--------|
| Forgetting measurements | Add `qc.measure()` or `qc.measure_all()` |
| `[(qc)]` without comma | Use `[(qc,)]` with trailing comma |
| Unbound parameters | Call `qc.assign_parameters()` first |
| Wrong import | Check local vs runtime package |
| Wrong result index | Use 0-based indexing: `result[0]` for first circuit |

### Memory Aids

🧠 **"Sampler Samples, Must Measure, Tuple Trails!"**

- **Sampler Samples** - Gets measurement outcomes (bitstrings)
- **Must Measure** - Requires `measure()` in circuit
- **Tuple Trails** - PUB format needs trailing comma `(qc,)`

🧠 **"Really Intelligent Developers Memorize Get-counts"**
- **R**esult `[i]` 
- **I**ndex for circuit
- **D**ata 
- **M**eas (measurement register)
- **G**et_counts()

→ `result[i].data.meas.get_counts()`

---

## 🎓 Section Summary

**Section 5: Sampler Primitive (12% of Exam)**

```
✅ Sampler = measurement counts and probabilities
✅ PUB format: [(circuit,)] with trailing comma!
✅ Sampler REQUIRES measurements in circuit
✅ result[0].data.meas.get_counts() for results
✅ StatevectorSampler for simulation, SamplerV2 for hardware
✅ Multiple circuits can be run simultaneously
✅ 12% of exam - MASTER THIS!
```

---

**Next**: Section 6 (Estimator) for expectation values! 🚀📊